In [1]:
import requests
import pandas as pd
from datetime import datetime
from sqlite_functions_test import *
import re

database = "market_data.db"

In [2]:
# Set the CoinGecko API URL for historical data
# https://www.coingecko.com/en/api/documentation

url = 'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart'

# Define a function to fetch historical data for a given cryptocurrency
# Pulls price, market cap, and total volume data and appends to a dataframe
def fetch_crypto_data(days, coin_id, interval='daily'):
    parameters = {
        'vs_currency': 'usd',
        'days': days,
        'interval': interval
    }
    
    response = requests.get(url.format(coin_id=coin_id), params=parameters)
    data = response.json()
    
    if response.status_code == 200:
        df_prices = pd.DataFrame(data['prices'], columns=['timestamp', f'{coin_id}_Price'])
        df_market_caps = pd.DataFrame(data['market_caps'], columns=['timestamp', f'{coin_id}_Market_Cap'])
        df_total_volumes = pd.DataFrame(data['total_volumes'], columns=['timestamp', f'{coin_id}_Volume'])
        
        df = pd.merge(df_prices, df_market_caps, on='timestamp', how='outer')
        df = pd.merge(df, df_total_volumes, on='timestamp', how='outer')
        
        df['Date'] = pd.to_datetime(df['timestamp'], unit='ms')
        #df['date'] = df['timestamp'].dt.strftime('%m/%d/%Y')
        df.set_index('Date', inplace=True)
        df.drop(columns=['timestamp'], inplace=True)
        
        return df
    else:
        print(f"Error: {response.status_code}")
        return None

In [3]:
# Fetch historical data for Bitcoin (BTC) and Dogecoin (DOGE)
num_days = '1200' # 1/1/2020 to 04/13/2023
coins = ['binancecoin','bitcoin', 'cardano', 'dogecoin', 'ethereum', 'polkadot', 'ripple', 'solana', 'uniswap']
today = datetime.now().strftime('%m/%d/%Y')

merged_data = None

for coin in coins:
    coin_df = fetch_crypto_data(num_days, coin_id=coin)
    
    if merged_data is None:
        merged_data = coin_df
    else:
        merged_data = pd.merge(merged_data, coin_df, on='Date', how='outer')

# Display the merged data
merged_data.dropna(how='any',axis=0, inplace=True) 
merged_data = merged_data[merged_data.index != today]

merged_data


,binancecoin_Price,binancecoin_Market_Cap,binancecoin_Volume,bitcoin_Price,bitcoin_Market_Cap,bitcoin_Volume,cardano_Price,cardano_Market_Cap,cardano_Volume,dogecoin_Price,...,polkadot_Volume,ripple_Price,ripple_Market_Cap,ripple_Volume,solana_Price,solana_Market_Cap,solana_Volume,uniswap_Price,uniswap_Market_Cap,uniswap_Volume
Date,,,,,,,,,,,,,,,,,,,,,
2020-09-17,27.855081,4.122358e+09,7.117046e+08,10952.249969,2.025240e+11,2.445909e+10,0.091295,2.842676e+09,6.814108e+08,0.002820,...,2.720708e+08,0.247217,1.113522e+10,1.770612e+09,2.549465,9.407010e+07,2.479533e+07,3.443832,2.199132e+08,1.802085e+09
2020-09-18,26.791783,3.959528e+09,4.251051e+08,10937.996397,2.023585e+11,2.354717e+10,0.094036,2.926254e+09,6.551521e+08,0.002816,...,2.995307e+08,0.251881,1.138531e+10,2.004021e+09,2.919199,1.073292e+08,2.179198e+07,3.443832,2.199132e+08,1.802085e+09
2020-09-19,27.095158,3.976467e+09,4.063788e+08,10927.150310,2.020913e+11,2.068177e+10,0.090861,2.816776e+09,5.855261e+08,0.002781,...,2.188458e+08,0.250086,1.126447e+10,1.869517e+09,3.084080,1.153982e+08,2.253326e+07,7.097694,7.328221e+08,5.688408e+09
2020-09-20,27.202935,4.024066e+09,3.506124e+08,11083.998361,2.049943e+11,1.833427e+10,0.091568,2.849153e+09,5.510882e+08,0.002787,...,1.807937e+08,0.251888,1.135006e+10,1.705001e+09,3.014729,1.130662e+08,1.094376e+07,5.702061,6.472084e+08,2.050756e+09
2020-09-21,26.429361,3.891106e+09,3.702732e+08,10923.326668,2.020325e+11,2.016173e+10,0.089246,2.788163e+09,5.607287e+08,0.002772,...,2.774053e+08,0.246396,1.110889e+10,1.633406e+09,2.867958,1.074852e+08,9.294740e+06,5.256579,6.324213e+08,1.316669e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-10,313.011277,4.944804e+10,4.176328e+08,28351.236994,5.485686e+11,1.142903e+10,0.389867,1.366884e+10,2.222585e+08,0.083369,...,1.456954e+08,0.505905,2.615316e+10,7.068495e+08,20.304337,7.886462e+09,1.956527e+08,5.916503,4.462994e+09,6.978628e+07
2023-04-11,318.226319,5.027828e+10,5.710393e+08,29657.974137,5.740596e+11,1.747538e+10,0.397182,1.391433e+10,2.743312e+08,0.084851,...,1.571155e+08,0.518237,2.680857e+10,1.057662e+09,20.875123,8.111385e+09,2.430511e+08,6.017906,4.541749e+09,7.682145e+07
2023-04-12,322.731167,5.098841e+10,1.121319e+09,30260.936109,5.853593e+11,1.832030e+10,0.401664,1.407415e+10,4.373187e+08,0.084254,...,2.122064e+08,0.517487,2.675076e+10,1.328973e+09,23.005655,9.019418e+09,1.081779e+09,5.942767,4.479515e+09,7.376588e+07


In [4]:
#convert wide data to long data
long_data = merged_data.reset_index()

# Convert using the melt function
long_data = long_data.melt(id_vars='Date', var_name='coin_metric', value_name='value')

# Extract the coin and metric names from the 'coin_metric' column
coin_metric_pattern = re.compile(r'(.+?)_(.+)')
long_data[['Coin', 'metric']] = long_data['coin_metric'].str.extract(coin_metric_pattern)

# Drop coin_metric column
long_data.drop(columns=['coin_metric'], inplace=True)

# Pivot the long format data
pivoted_data = long_data.pivot_table(index=['Date', 'Coin'], columns='metric', values='value').reset_index()

# Rename the columns
pivoted_data.columns.name = None

# Reorder the columns to the desired output format
pivoted_data = pivoted_data[['Date', 'Coin', 'Price', 'Volume', 'Market_Cap']]

# pivoted_data = pivoted_data.rename(columns={
#     'date': 'Date',
#     'coin': 'Coin',
#     'price': 'Price',
#     'total_volume': 'Volume',
#     'market_cap': 'Market_Cap'
# })

# save as Unix time for compatibility with Tableau??
#pivoted_data['date'] = pd.to_datetime(pivoted_data['date'], format='%m/%d/%Y')
#pivoted_data['date'] = pivoted_data['date'].apply(lambda x: int(x.timestamp()))


pivoted_data

,Date,Coin,Price,Volume,Market_Cap
0,2020-09-17,binancecoin,27.855081,7.117046e+08,4.122358e+09
1,2020-09-17,bitcoin,10952.249969,2.445909e+10,2.025240e+11
2,2020-09-17,cardano,0.091295,6.814108e+08,2.842676e+09
3,2020-09-17,dogecoin,0.002820,1.553926e+08,3.560026e+08
4,2020-09-17,ethereum,365.130551,1.329820e+10,4.113618e+10
...,...,...,...,...,...
8455,2023-04-14,ethereum,2012.785415,1.395640e+10,2.422845e+11
8456,2023-04-14,polkadot,6.644896,2.046325e+08,8.125660e+09
8457,2023-04-14,ripple,0.513900,1.078338e+09,2.659469e+10
8458,2023-04-14,solana,24.427710,6.660668e+08,9.566763e+09


In [5]:
#update coin names
coin_mapping = {
    'binancecoin': 'Binance - BNB',
    'bitcoin': 'Bitcoin - BTC',
    'cardano': 'Cardano - ADA',
    'dogecoin': 'Dogecoin - DOGE',
    'ethereum': 'Ethereum - ETH',
    'polkadot': 'Polkadot - DOT',
    'ripple': 'Ripple - XRP',
    'solana': 'Solana - SOL',
    'uniswap': 'Uniswap - UNI'
    # Add more mappings if needed
}

pivoted_data['Coin'] = pivoted_data['Coin'].replace(coin_mapping)

In [9]:
pivoted_data

,Date,Coin,Price,Volume,Market_Cap
0,2020-09-17,Binance - BNB,27.855081,7.117046e+08,4.122358e+09
1,2020-09-17,Bitcoin - BTC,10952.249969,2.445909e+10,2.025240e+11
2,2020-09-17,Cardano - ADA,0.091295,6.814108e+08,2.842676e+09
3,2020-09-17,Dogecoin - DOGE,0.002820,1.553926e+08,3.560026e+08
4,2020-09-17,Ethereum - ETH,365.130551,1.329820e+10,4.113618e+10
...,...,...,...,...,...
8455,2023-04-14,Ethereum - ETH,2012.785415,1.395640e+10,2.422845e+11
8456,2023-04-14,Polkadot - DOT,6.644896,2.046325e+08,8.125660e+09
8457,2023-04-14,Ripple - XRP,0.513900,1.078338e+09,2.659469e+10
8458,2023-04-14,Solana - SOL,24.427710,6.660668e+08,9.566763e+09


In [6]:
# Create the Bitcoin price table if it doesn't exist
# conn = create_connection(database)

# if conn is not None:
#     sql_create_table =  """
#     CREATE TABLE IF NOT EXISTS market_data_plz (  
#         Date TIMESTAMP,
#         Coin TEXT,
#         Price REAL,
#         Volume INTEGER,
#         Market_Cap REAL);
#     """
#     create_table(conn, sql_create_table)
#     conn.close()
# else:
#     print("Error! Cannot create the database connection.")

In [10]:
conn = create_connection(database)

# Insert the DataFrame into the 'market_data_plz' table
pivoted_data.to_sql('Crypto_Data', conn, if_exists='replace', index=False)

conn.close()